In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Критерий Хи-квадарт позволяет нам сравнить отношения количества успеха к неуспехам в таблицах

In [12]:
url = URL(
    user="ABUHTAREVICH",
    password="/",
    account="/",
    warehouse="COMPUTE_WH",
    database="SEVEREX",
    schema="/",
    #role='xxxxx',
    #authenticator='https://xxxxx.okta.com',
)
engine = create_engine(url)
connection = engine.connect()

In [13]:
query_checking = '''
with t1 as (
select *,
  datediff(day, date(INSTALL_TIME), date(EVENT_TIME)) as day,
  max(event_value:group_name::string) over(partition by appsflyer_id) as group_name,
  max(event_value:test_name::string) over(partition by appsflyer_id) as test_name
   from t_events
where 
    APP_ID = 'id1265751053'
   and date(EVENT_TIME) between '2022-08-09' and '2022-11-02'
   and date(INSTALL_TIME) between '2022-08-09' and '2022-10-29'
),
t2 as(
select appsflyer_id, 
       group_name,
       iff(max(event_name = 'trial_started') over(partition by appsflyer_id), 1,0) as conversion
from t1
where group_name in ('Control', 'Variant_A', 'Variant_B')
and test_name = 'Landing_7UK'
order by 1
)

select distinct APPSFLYER_ID,
       group_name,
       conversion
from t2
'''

In [14]:
checking = pd.read_sql(query_checking, connection)

In [ ]:
#создали результаты теста конверсии

In [15]:
checking.head()

,appsflyer_id,group_name,conversion
0,1661086879432-7974553,Variant_B,0
1,1661087486104-9146464,Control,0
2,1661094432574-2868054,Control,0
3,1661094741056-2438703,Variant_A,0
4,1661094791534-0107133,Control,0


In [ ]:
#Составляем таблицу сопряженности, при сравнении более двух таблиц не забыть убрать поправку Йетса

In [16]:
table = pd.crosstab(
    checking['group_name'],
    checking['conversion'],
    margins = True
)
table

table1 = pd.crosstab(
    checking['group_name'],
    checking['conversion']
)
table

conversion,0,1,All
group_name,,,
Control,1186,75,1261
Variant_A,1272,98,1370
Variant_B,1224,99,1323
All,3682,272,3954


In [ ]:
#воспользуемся библиотекой scipy для расчета Хи-квадрат критерия и p-value

In [17]:
stats.chi2_contingency(table1)

(2.6217930065187356,
 0.2695782702219307,
 2,
 array([[1174.2544259 ,   86.7455741 ],
        [1275.75619626,   94.24380374],
        [1231.98937785,   91.01062215]]))

In [18]:
stat, p_val, df, ef = stats.chi2_contingency(table1)

print (p_val)

0.2695782702219307


In [ ]:
#p-value > 0.05, а значит принимаем нулевую гипотезу, что результаты теста незначительны. Контрольная группа победила 